<a href="https://colab.research.google.com/github/KyoungmiKwon/Bigdata_Training_at_ITwill/blob/main/Team_Project/C05_KM_TextPreprogressing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mecab 설치

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [ ]:
%env JAVA_HOME "/user/lib/jvm/java-8-openjdk-amd64"

In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

In [ ]:
!git clone https://github.com/entelecheia/eKoNLpy.git

In [ ]:
!pip install /content/eKoNLpy

# import

In [6]:
import konlpy
from ekonlpy.tag import Mecab

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import logging # word2vec 학습과정에서 로그 메세지를 양식에 맞는 info 수준으로 볼 수 있다.
from gensim.models import word2vec

from sklearn.feature_extraction.text import TfidfVectorizer #TF-IDF
from gensim.models.doc2vec import TaggedDocument,Doc2Vec

from sklearn.model_selection import train_test_split, cross_validate, cross_val_score

from sklearn.neighbors import KNeighborsClassifier 

from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV # 하이퍼 파라미터 튜닝
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, classification_report

# 뉴스 기사 불러오기

In [39]:
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Project/samsung_only.xlsx',sheet_name='all',header=0)

In [40]:
data[:1]

,No,Code,Data,Journal,Title,Text,URL,Sample Y/N,In Charge,s
0,52,5930,2021.04.13 17:50,파이낸셜뉴스,"삼성전자 사내식당 2곳, 신세계·풀무원이 운영한다",삼성전자가 사내식당 2곳에 대해 실시한 외부 급식업체 경쟁입찰에서 '신세계푸드'와 ...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1


In [41]:
len(data)

1714

# 데이터 정제

In [42]:
# 중복기사 제거
data = data.drop_duplicates(subset=['Data', 'Text'], keep='first')

In [46]:
len(data)

1688

In [47]:
data.iloc[0]['Text'].split(sep='@')[0]

'삼성전자가 사내식당 2곳에 대해 실시한 외부 급식업체 경쟁입찰에서 \'신세계푸드\'와 \'풀무원푸드앤컬처\'가 운영 업체로 선정됐다고 13일 밝혔다.    삼성전자는 지난 2월, 단체급식의 다양한 메뉴와 향상된 서비스를 통해 임직원의 복리후생을 향상시키고자 수원사업장과 기흥사업장 내 사내식당 2곳에 대해 공개 입찰을 공고했다.    약 20개 업체가 참여한 이번 입찰은 서류 심사를 거쳐 △메뉴 구성과 서비스 등을 평가하는 프레젠테이션 △업체의 인프라와 위생 등을 점검하는 현장 실사 △임직원 음식 품평회까지 총 3단계에 걸친 엄격하고 공정한 절차를 통해 진행됐다. 기존에 단체급식을 공급했던웰스토리는 이 입찰에서 배제됐다.    최종 선정된 \'신세계푸드\'와 \'풀무원푸드앤컬처\'는 약 한달 반 동안 고용승계, 업무인수 등을 마친 후 6월 1일부터 운영을 시작할 예정이다. 삼성전자 관계자는 "신규 업체에 대한 임직원 만족도와 운영상 보완점 등을 검토 후에 다른 사내식당에 대해서도 경쟁입찰을 확대해 나갈 계획"이라고 말했다.  ahnman'

In [137]:
# 기사 본문 기자의 이메일 OOOO @ asdfas.com 에서 @ 전 까지의 내용으로 저장
for i in range(len(data)):
    data.iloc[i]['Text'] = data.iloc[i]['Text'].split(sep='@')[0]

# 파일 새로 저장
data.to_excel('/content/drive/MyDrive/Colab Notebooks/Project/article.xlsx',index=False)
# 다시 불러 오기
news = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Project/article.xlsx',header=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [138]:
#> 1. 기사에서 한글 제외하고 모두 제거
news['Text'] = news['Text'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]","") 

#> 2. 기사에서 특정품사 추출
tag_cls = ['NNG','VV+EC','XSV+EP']
# NNG = 일반명사, VV = 동사, EC = 연결어미, XSV = 동사파생접미사, EP = 선어말 어미, SL = 외국어

mc = Mecab()
artc = []
for line in news['Text'][:]:
    temp = []  
    for word in mc.pos(line):
        if word[1] in tag_cls:
            temp.append(word[0])
    artc.append(temp)

#> 3. 불용어처리
swords = ['삼성전자','삼성','기자','인텔','신세계','풀무원','마음속','파이낸셜','뉴스','조선','비즈','조선비즈', '자세', '내용', '세계', '단전', '아시아','이미지', '출처', '머니','속보',
          '저작','무단','전재','배포','금지','월일','기자', '전자','저작권','종합', '정보', '미디어', '유준','낸드','닷컴']

text = []

for tx in artc:
    temp = []
    for word in tx:
        if word not in swords:
            if len(word) >1 :       #4) 길이가 짧은 글자 제거 (optional)
                temp.append(word)
    text.append(temp)

#> Word2Vec 에서 빈도수 낮은 단어를 삭제 시키는 파라미터가 있음

#> 5. 낮은 빈도수 단어 제거
# step 1) 단어 별 빈도수
## 단어의 빈도 수
vocab ={}
for word in text:
    for w in word:
        if w not in vocab:
            vocab[w] = 0
        vocab[w] += 1



# step 2) 빈도수 낮은 단어의 삭제
## 1. 빈도수가 높은 순서대로 정렬
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse =False)
print(vocab_sorted)

## 2. 빈도수가 낮은 순서대로 정수 인덱스 부여
word_to_index = {}

for (word,freq) in vocab_sorted:
    word_to_index[word] = freq
print(word_to_index)

# 3. 빈도수 낮은 단어의 선택
words_del = {}
threshold = 30
for word,cnt in word_to_index.items():
    if cnt < threshold:
        words_del[word]= cnt
print(words_del)

# 4. 빈도 낮은 단어 삭제
txt = []

for tx in text:
    temp = []
    for word in tx:
        if word not in words_del:
            temp.append(word)
    txt.append(temp)


[('이관', 1), ('속력', 1), ('전년동기비', 1), ('순매도액', 1), ('학습효과', 1), ('대훈', 1), ('비농업부문', 1), ('테이퍼링', 1), ('연방은행총재', 1), ('당삼', 1), ('부실채권', 1), ('여수신', 1), ('간주', 1), ('은행업', 1), ('공시방법', 1), ('신용공여', 1), ('정책위', 1), ('규준', 1), ('야망', 1), ('원합', 1), ('협의안', 1), ('경조사', 1), ('목돈', 1), ('살림', 1), ('그릴', 1), ('알뜰', 1), ('새살림', 1), ('발품', 1), ('손품', 1), ('파이어', 1), ('파이어폭스', 1), ('광고시장', 1), ('자년', 1), ('업소', 1), ('공구', 1), ('휴머노이드', 1), ('장향', 1), ('메탄', 1), ('유건', 1), ('소재제조업체', 1), ('개선책', 1), ('지양', 1), ('생애', 1), ('귀재', 1), ('선구안', 1), ('영호', 1), ('초대형주', 1), ('여관', 1), ('청바지', 1), ('금광', 1), ('의복', 1), ('여간', 1), ('시공권', 1), ('흑석', 1), ('재개발사업', 1), ('부대', 1), ('복리시설', 1), ('지로', 1), ('순환고', 1), ('계초', 1), ('암산', 1), ('자연공원', 1), ('관리비', 1), ('분담금', 1), ('이주비대출', 1), ('주안', 1), ('주공', 1), ('스카이라운지', 1), ('명품', 1), ('마감재', 1), ('지하', 1), ('대실', 1), ('명작', 1), ('승전보', 1), ('수목', 1), ('신반', 1), ('포차', 1), ('주구', 1), ('공방', 1), ('크로스라이선스', 1), ('페트병', 1), ('폴리', 1), ('스티로폼', 1), ('정호', 1), (

In [146]:
len(txt)

1688

In [147]:
news_dt = news.copy()

In [148]:
news_dt['seperate'] = txt

In [149]:
# 내용의 길이가 너무 짧은 기사의 전체 내역 삭제

line_del = []
for i,line in enumerate(news_dt['seperate']):
    if len(line) < 7:
        line_del.append(i)
        news_dt = news_dt.drop(index=i, axis=0)

print(len(line_del))
print(len(news_dt))


17
1671


In [130]:
news_dt[:1]

,No,Code,Data,Journal,Title,Text,URL,Sample Y/N,In Charge,s,seperate
0,52,5930,2021.04.13 17:50,파이낸셜뉴스,"삼성전자 사내식당 2곳, 신세계·풀무원이 운영한다",삼성전자가 사내식당 곳에 대해 실시한 외부 급식업체 경쟁입찰에서 신세계푸드와 풀무원...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1,"[사내, 식당, 실시, 외부, 급식, 업체, 경쟁입찰, 푸드, 컬처, 운영, 업체,..."


In [131]:
news_dt.to_excel('/content/drive/MyDrive/Colab Notebooks/Project/news_dt.xlsx',index=False)

In [145]:
for i in news_dt['seperate'][:3]:
    

['사내', '식당', '실시', '외부', '급식', '업체', '경쟁입찰', '푸드', '컬처', '운영', '업체', '선정', '단체', '급식', '다양', '메뉴', '향상', '서비스', '임직원', '복리', '후생', '향상', '사업장', '기흥', '사업장', '사내', '식당', '공개입찰', '공고', '업체', '참여', '이번', '입찰', '서류', '심사', '메뉴', '구성', '서비스', '평가', '프레젠테이션', '업체', '인프라', '위생', '점검', '현장실사', '임직원', '음식', '품평회', '단계', '엄격', '공정', '절차', '진행', '기존', '단체', '급식', '공급', '스토리', '입찰', '배제', '최종', '선정', '푸드', '컬처', '동안', '고용', '승계', '업무', '인수', '운영', '시작', '예정', '관계자', '신규', '업체', '임직원', '만족도', '운영', '보완', '검토', '사내', '식당', '경쟁입찰', '계획']
['대통령', '현지', '시간', '공급망', '강화', '온라인', '회담', '참석', '인사말', '유튜브', '실시간', '중계', '백악관', '전세계', '공급', '부족', '의견', '가운데', '이번', '회담', '결정', '발표', '백악관', '대변인', '현지', '시간', '정례', '브리핑', '대통령', '회의', '참석', '이유', '의견', '문제', '지속', '논의', '일부', '설명', '산업', '협력', '의회', '합의', '부족', '사태', '방지', '국가', '산업', '차원', '합의', '이날', '백악관', '국가', '안보', '보좌관', '국가경제위원회', '주재', '부족', '해소', '온라인', '화상회의', '대통령', '합류', '일정', '참석', '인사말', '이날', '백악관', '초대', '포함', '대만', '구글', '모회사', '알파벳', '테크놀

In [144]:
for i in news_dt['seperate'][:3]:
    for x in i:
        print(x)

사내
식당
실시
외부
급식
업체
경쟁입찰
푸드
컬처
운영
업체
선정
단체
급식
다양
메뉴
향상
서비스
임직원
복리
후생
향상
사업장
기흥
사업장
사내
식당
공개입찰
공고
업체
참여
이번
입찰
서류
심사
메뉴
구성
서비스
평가
프레젠테이션
업체
인프라
위생
점검
현장실사
임직원
음식
품평회
단계
엄격
공정
절차
진행
기존
단체
급식
공급
스토리
입찰
배제
최종
선정
푸드
컬처
동안
고용
승계
업무
인수
운영
시작
예정
관계자
신규
업체
임직원
만족도
운영
보완
검토
사내
식당
경쟁입찰
계획
대통령
현지
시간
공급망
강화
온라인
회담
참석
인사말
유튜브
실시간
중계
백악관
전세계
공급
부족
의견
가운데
이번
회담
결정
발표
백악관
대변인
현지
시간
정례
브리핑
대통령
회의
참석
이유
의견
문제
지속
논의
일부
설명
산업
협력
의회
합의
부족
사태
방지
국가
산업
차원
합의
이날
백악관
국가
안보
보좌관
국가경제위원회
주재
부족
해소
온라인
화상회의
대통령
합류
일정
참석
인사말
이날
백악관
초대
포함
대만
구글
모회사
알파벳
테크놀로지
피스톤
그룹
워터
테크놀로지
개사
포함
사장
참석
이날
대통령
부족
문제
해결
의회
지지
공급망
강화
달러
규모
인프라
연구개발
관련
일자리
계획
강조
백악관
이중
달러
배정
제안
실행
시간
중단기
해결
방안
모색
자동차
전세계
부족
자동차
생산량
만대
방해
경고
주말
트럭
교대
근무
취소
미래
신성장동력
이란
관측
강세
사업부
완전
철수
확정
로봇
주력
로봇
전담팀
신설
상태
오후
거래일
거래
월부
철수
미래
먹거리
전기자동차
부품
로봇
공학
인공지능
집중
이란
제기
설립
로봇
센터
비즈니스
솔루션
본부
이관
본부
영업
인프라
역량
활용
로봇
진출
교두보
마련
계획
로봇
제조
전문기업
지분
인수
로봇
산업
개발
속력
소비자
가전
직속
로봇
전담팀
신설
과거
통합
연구
조직
리서치
로봇
연구
진행
부문
신설
로봇
상용화
본격
속도
한편
로봇
제조사
지분
이후
협업
로봇
플랫폼
매출
계획
